In [95]:
import numpy as np

In [96]:
a = np.array([])

In [451]:
def generate_TF_inner(arr, n):
        if n == 0:
            return arr
        else:
            new_arr = []
            for i in arr:
                for possible_value in (True, False):
                    new_i = i.copy()
                    new_i.append(possible_value)
                    new_arr.append(new_i)
            if (len(new_arr) == 0):
                new_arr = [[True],[False]]
            return generate_TF_inner(new_arr, n-1)
                    
# generate all possible values of True, False for n variables
def generate_TF(n):
    arr = []
    ans = generate_TF_inner(arr, n)
    return ans

In [20]:
def simplifyString(string):
    return string.replace(" ", "")
def fill_vars(string):
    vars = []
    for i in string:
        if (re.match('[a-zA-Z]', i)):
            vars.append(i)
    return vars
def fill_ops(string):
    ops = []
    for i in string:
        if (i in available_ops['parenthesis'] or \
            i in available_ops['binary'] or \
            i in available_ops['unary']):
            ops.append(i)
    return ops


Парсим выражение.
Есть операторы. Есть формулы. Формулы - это либо атомы, либо выражения. Объединение операторов и других формул даёт новую формулу.

Видим выражение (A&B)|(!A&(B|C))&D

1. Атом - инстанс класса, который хранит в себе номер аргумента из комбинации значений всех атомов в рамках одной интерпретации.
2. Делаем сет всех атомов.
3. Дробим исходное выражение на формулы. 
    1. Сначала отсекаем части по операторам (то есть что между скобок) - в рамках одного уровня заносим все встреченные операции. ['|', '&']
    2. Вместо скобок создаём формулы. [(A&B), (!A&(B|C), D]
    4. Эти штуки рекурсивно преобразуем по той же схеме. 
    5. После преобразования каждая штука должна возвратить формулу куда можно подставить комбинацию значений.
    6. Полученные формулы объединеям в одну с учетом стека операций и возвращаем.
4. Получится большая лямбда, которая принимает комбинацию всех значений.
5. Делаем таблицу.



In [349]:
class Atom:
    def __init__(self, name):
        self.name = name
    def substitute(self, args):
        return args[self.name]
    def __str__(self):
        return "Atom " + self.name
    def __repr__(self):
        return "Atom " + self.name

In [385]:
def create_atom_set(expression):
    atom_set = {}
    for i in expression:
        if i not in atom_set and re.match('[a-zA-Z]', i):
            atom_set[i] = Atom(i)
            print(i)
    return atom_set

In [371]:
def evaluate_expr(expression, atom_set):
    ops_stack = []
    sub_expr_stack = []
    parenth_stack = []
    lambdas_stack = []
    available_ops = ['&', '|', '!']
    begin = 0
    for char_index in zip(expression, range(0, len(expression))):
        i = char_index[1]
        char = char_index[0]
        if (char == '(' and len(parenth_stack) == 0):
            parenth_stack.append('(')
            begin = i
        elif (char == '('):
            parenth_stack.append('(')
        elif (char == ')' and len(parenth_stack) == 1):
            parenth_stack.pop()
            sub_expr_stack.append(expression[begin+1:i])
            begin = i+1
        elif (char == ')'):
            parenth_stack.pop()
        elif (char in available_ops and len(parenth_stack) == 0):
            print('added op', char)
            ops_stack.append(char)
        elif (re.match('[a-zA-Z]', char)):
            print('match')
            if (len(ops_stack) == 0):
                print('added var', char)
                sub_expr_stack.append(char)
            else:
                last_op = ops_stack[-1]
                if (last_op == '!'):
                    char_loc = char
                    print('neg op on char', char_loc)
                    lambdas_stack.append(lambda x: not atom_set[char_loc].substitute(x))
                    ops_stack.pop()
                elif (last_op == '&'):
                    print('and op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
                elif (last_op == '|'):
                    print('or op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
    return {'ops': ops_stack, 'sub_expr': sub_expr_stack, 'lambdas': lambdas_stack}

In [355]:
l = [1, 2, 3]
l[3:]

[]

In [361]:
split_for_formulas(expression)

{'ops': ['|', '&'], 'sub_expr': ['A&B', '!A&(B|C)']}

In [415]:
# simple expr
simple_expr = '!A&B|C'
simple_expr_atom_set = atom_set(simple_expr)
simple_args = {'A': True, 'B': True, 'C': True}
stacks = evaluate_expr(simple_expr, simple_expr_atom_set)

A
B
C
added op !
match
neg op on char A
added op &
match
and op on char B
added op |
match
or op on char C


In [416]:
/print stacks['lambdas'][0](simple_args)
/print stacks['lambdas'][1](simple_args)
/print stacks['lambdas'][2](simple_args)

False
True
True


In [417]:
s_f_lambda = full_lambda_on_stacks(stacks['ops'], stacks['lambdas'])

In [418]:
s_f_lambda(simple_args)

True

In [419]:
simple_all_args = generate_TF(3)

[[True, True, True], [True, True, False], [True, False, True], [True, False, False], [False, True, True], [False, True, False], [False, False, True], [False, False, False]]


In [420]:
table = []
for i in simple_all_args:
    new_comb = {}
    for char_with_index in zip(simple_expr_atom_set, range(0, len(simple_expr_atom_set))):
        index = char_with_index[1]
        char = char_with_index[0]
        new_comb[char] = i[index]
    table.append(new_comb)

In [421]:
table

[{'A': True, 'B': True, 'C': True},
 {'A': True, 'B': True, 'C': False},
 {'A': True, 'B': False, 'C': True},
 {'A': True, 'B': False, 'C': False},
 {'A': False, 'B': True, 'C': True},
 {'A': False, 'B': True, 'C': False},
 {'A': False, 'B': False, 'C': True},
 {'A': False, 'B': False, 'C': False}]

In [422]:
for i in table:
    ans = s_f_lambda(i)
    print(i, "is", ans, end="\n")

{'A': True, 'B': True, 'C': True} is True
{'A': True, 'B': True, 'C': False} is False
{'A': True, 'B': False, 'C': True} is True
{'A': True, 'B': False, 'C': False} is False
{'A': False, 'B': True, 'C': True} is True
{'A': False, 'B': True, 'C': False} is True
{'A': False, 'B': False, 'C': True} is True
{'A': False, 'B': False, 'C': False} is False


for expression: '!A&B|C'


In [354]:
def build_binary_lambda(op, lambda1, lambda2):
    if (op == '&'):
        def and_(x):
            return lambda1(x) and lambda2(x)
        return and_
    elif (op == '|'):
        def or_(x):
            return lambda1(x) or lambda2(x)
        return or_

In [431]:
def full_lambda_on_stacks(ops_stack, lambda_stack):
    lambda_stack_copy = lambda_stack.copy()
    current_lambda = []
    if (len(ops_stack) == 0 and len(lambda_stack_copy) == 1):
        return lambda_stack_copy[0]
    for i in range(0, len(ops_stack)):
        if (i == 0):
            current_lambda = build_binary_lambda(ops_stack[i] ,lambda_stack_copy[0], lambda_stack_copy[1])
            lambda_stack_copy = lambda_stack_copy[2:]
        else:
            current_lambda = build_binary_lambda(ops_stack[i], current_lambda, lambda_stack_copy[0])
            lambda_stack_copy = lambda_stack_copy[1:]
    return current_lambda

In [455]:
def lambda_for(expression, atom_set):
    # build variables dictionary
    if (atom_set == None):
        atom_set = create_atom_set(expression)
    ops_stack = []
    parenth_stack = []
    lambdas_stack = []
    available_ops = ['&', '|', '!']
    begin = 0
    for char_index in zip(expression, range(0, len(expression))):
        i = char_index[1]
        char = char_index[0]
        if (char == '(' and len(parenth_stack) == 0):
            parenth_stack.append('(')
            begin = i
        elif (char == '('):
            parenth_stack.append('(')
        elif (char == ')' and len(parenth_stack) == 1):
            parenth_stack.pop()
            inner_expr = expression[begin+1:i]
            # parse inner expression and build lambda
            inner_lambda = lambda_for(inner_expr, atom_set)
            print(inner_lambda)
            last_op = ops_stack[-1]
            if (last_op == '!'):
                lambdas_stack.append(lambda x: not inner_lambda(x))
                ops_stack.pop()
            elif (last_op == '&'):
                lambdas_stack.append(inner_lambda)
            elif (last_op == '|'):
                lambdas_stack.append(inner_lambda)
            begin = i+1
        elif (char == ')'):
            parenth_stack.pop()
        elif (char in available_ops and len(parenth_stack) == 0):
            print('added op', char)
            ops_stack.append(char)
        elif (re.match('[a-zA-Z]', char) and len(parenth_stack) == 0):
            print('match')
            if (len(ops_stack) == 0):
                print('added var', char)
                lambdas_stack.append(atom_set[char].substitute)
            else:
                last_op = ops_stack[-1]
                if (last_op == '!'):
                    char_loc = char
                    print('neg op on char', char_loc)
                    lambdas_stack.append(lambda x: not atom_set[char_loc].substitute(x))
                    ops_stack.pop()
                elif (last_op == '&'):
                    print('and op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
                elif (last_op == '|'):
                    print('or op on char', char)
                    lambdas_stack.append(atom_set[char].substitute)
    # after loop collect the stacks
    full_lambda = full_lambda_on_stacks(ops_stack, lambdas_stack)
    return full_lambda

In [442]:
expression = '!A&B|(!A&C|(A&B))'

In [443]:
l = lambda_for(expression, None)
/print l

A
B
C
added op !
match
neg op on char A
added op &
match
and op on char B
added op |
added op !
match
neg op on char A
added op &
match
and op on char C
added op |
match
added var A
added op &
match
and op on char B
<function build_binary_lambda.<locals>.and_ at 0x7f9199f270d0>
<function build_binary_lambda.<locals>.or_ at 0x7f9199f27040>
<function build_binary_lambda.<locals>.or_ at 0x7f919b0f83a0>


In [445]:
l({'A': False, 'B': False, 'C': True})

True

In [448]:
def true_table_for(fun, var_names):
    table = []
    result_keyword = "result"
    all_combs = generate_TF(len(var_names))
    for i in all_combs:
        new_comb = {}
        for char_with_index in zip(var_names, range(0, len(var_names))):
            index = char_with_index[1]
            char = char_with_index[0]
            new_comb[char] = i[index]
        result = fun(new_comb)
        new_comb[result_keyword] = result
        table.append(new_comb)
    return table

In [449]:
true_table_for(l, ['A', 'B', 'C'])

[[True, True, True], [True, True, False], [True, False, True], [True, False, False], [False, True, True], [False, True, False], [False, False, True], [False, False, False]]


[{'A': True, 'B': True, 'C': True, 'result': True},
 {'A': True, 'B': True, 'C': False, 'result': True},
 {'A': True, 'B': False, 'C': True, 'result': False},
 {'A': True, 'B': False, 'C': False, 'result': False},
 {'A': False, 'B': True, 'C': True, 'result': True},
 {'A': False, 'B': True, 'C': False, 'result': True},
 {'A': False, 'B': False, 'C': True, 'result': True},
 {'A': False, 'B': False, 'C': False, 'result': False}]

In [456]:
expr2 = '!(A&B)'
l_2 = lambda_for(expr2, None)
true_table_for(l_2, ['A','B'])

A
B
added op !
match
added var A
added op &
match
and op on char B
<function build_binary_lambda.<locals>.and_ at 0x7f9199f27670>


[{'A': True, 'B': True, 'result': False},
 {'A': True, 'B': False, 'result': True},
 {'A': False, 'B': True, 'result': True},
 {'A': False, 'B': False, 'result': True}]